In [1]:
%load_ext Cython

In [2]:
%%cython

import os
import os.path
from libcpp cimport str

                
# shir bracya 207447319
# Ayelet Pollack 206935330
# 150060.21.5780.42

#global 
fileArray = []
#global
file1 = 0
#global
line = 2
label_count = 0
#global
classsymbolTable = None
methodsymbolTable = None
subroutinType = None

cdef class symbolTable:
    cdef int length;
    cdef dict table;
    cdef dict kind;
    cdef str temp;
    def __init__(self):
        self.length = 0
        self.table = dict()
        self.kind = {"argument":"0", "static":"0", "field":"0", "var":"0"}
    def restartTable(self):
        self.length = 0
        self.table = dict()
        self.kind = {"argument":"0", "static":"0", "field":"0", "var":"0"}
    def define(self,name1, kind1, type1):
        self.length = self.length + 1
        number = int(self.kind[kind1])
        self.table.update({self.length: [name1,type1,kind1,str(number)]})
        number1 = number + 1
        temp = str(number1)
        self.kind[kind1] = temp 
    def varCount(self, kind1):
        return self.kind[kind1]
    def kindOf(self, name1):
        for item in self.table:
            if (self.table[item][0]==name1):
                if (self.table[item][2] == "var"):
                    return "local"
                if (self.table[item][2] == "field"):
                    return "this"
                return self.table[item][2]
        return None
    def typeOf(self, name1):
        for item in self.table:
            if (self.table[item][0]==name1):
                return self.table[item][1]
        return None
    def indexOf(self, name1):
        for item in self.table:
            if (self.table[item][0]==name1):
                return self.table[item][3]
        return None
    def printMyTable(self):
        for item in self.table:
            print(self.table[item])
            print("\n")
            
cdef get_label():
    global label_count
    label = label_count
    label_count = label_count + 1
    return str(label)
            
cdef classFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    classsymbolTable = symbolTable()
    line = 4
    while (line < len(fileArray)) and ("field" in fileArray[line]) or ("static" in fileArray[line]):
        kind = fileArray[line].split(" ")[1]
        line = line + 1 
        type1 = fileArray[line].split(" ")[1]
        line = line + 1 
        name = fileArray[line].split(" ")[1]
        classsymbolTable.define(name,kind,type1)
        line = line + 1 # , / ;
        while (line < len(fileArray) and ("," in fileArray[line])):
            line = line + 1 
            name = fileArray[line].split(" ")[1]
            classsymbolTable.define(name,kind,type1)
            line = line + 1
        if (";" in fileArray[line]):
            line = line + 1


cdef subroutinfunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    global subroutinType
    #print(fileArray[line])
    while (line < len(fileArray) and "constructor" in fileArray[line] or "method" in fileArray[line] or "function" in fileArray[line]):
        subroutinType = fileArray[line].split(" ")[1]
        line = line + 1
        returnType = fileArray[line].split(" ")[1]
        line = line + 1
        subroutinName = fileArray[line].split(" ")[1]
        #print(fileArray[line])
        subroutinFuncTable(subroutinType,returnType, subroutinName)
        line = line + 1


cdef subroutinFuncTable(subroutinType ,returnType, subroutinName):
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    methodsymbolTable = symbolTable()
    if (subroutinType == "method"):
        classname = fileArray[2].split(" ")[1] 
        methodsymbolTable.define("this","argument",classname)
    line = line + 1 # (
    line = line + 1 # ), type
    if(not(")" in fileArray[line])):
        parameterList()
    subroutine_body(subroutinType ,returnType, subroutinName)
    
cdef varFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    while(line < len(fileArray) and ("var" in fileArray[line+1])):
        line = line + 1 # var
        line = line + 1 # type
        type1 = fileArray[line].split(" ")[1] # type
        line = line + 1
        name = fileArray[line].split(" ")[1] # id
        methodsymbolTable.define(name,"var",type1)
        while ("," in fileArray[line+1]):
            line = line + 1 # ,
            line = line + 1 # id
            name = fileArray[line].split(" ")[1] 
            methodsymbolTable.define(name,"var",type1)
        if (";" in fileArray[line+1]):
            line = line + 1
       
    
cdef parameterList():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    while (line < len(fileArray) and not(")" in fileArray[line])):
        type1 = fileArray[line].split(" ")[1]
        line = line + 1
        name = fileArray[line].split(" ")[1]
        methodsymbolTable.define(name,"argument",type1)
        line = line + 1 # ,\ )
        if ("," in fileArray[line]):
            line = line + 1
    
cdef subroutine_body(subroutinType ,returnType, subroutinName):
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    line = line + 1 # {
    varFunc()
    #methodsymbolTable.printMyTable()
    classname = fileArray[2].split(" ")[1] 
    varCount = methodsymbolTable.varCount("var")
    file1.write("function "+classname+"."+subroutinName+" "+varCount+"\n")
    if (subroutinType == "constructor"):
        fieldCount = classsymbolTable.varCount("field")
        file1.write("push constant "+fieldCount+"\n")
        file1.write("call Memory.alloc 1"+"\n")
        file1.write("pop pointer 0"+"\n") 
    elif (subroutinType == "method"):
        file1.write("push argument 0\n")
        file1.write("pop pointer 0"+"\n")
    statment()
    line = line + 1
    
    
cdef statment():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    flag = True
    while(flag): 
        #print(fileArray[line+1])
        if ("let" in fileArray[line+1]):
            letFunc()
        elif ("do" in fileArray[line+1]):
            doFunc()
        elif (" if " in fileArray[line+1]):
            ifFunc()
        elif ("while" in fileArray[line+1]):
            whileFunc()
        elif ("return" in fileArray[line+1]):
            returnFunc()
        else:
            flag = False
        
cdef ifFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    line = line + 1 # if
    line = line + 1 # (
    expressionfunc()
    line = line + 1 # )
    line = line + 1 # {
    false_label = get_label()
    end_label = get_label()
    file1.write("if-goto IF_TRUE"+false_label+"\n")
    file1.write("goto IF_FALSE"+end_label+"\n")
    file1.write("label IF_TRUE"+false_label+"\n")
    statment()
    line = line + 1 # }
    if ("else" in fileArray[line+1]):
        line = line + 1 # else
        temp = get_label()
        file1.write("goto IF_END"+temp+"\n")
        file1.write("label IF_FALSE"+end_label+"\n")
        line = line + 1# {
        statment()
        file1.write("label IF_END"+temp+"\n")
        line = line + 1 #}
    else:
        file1.write("label IF_FALSE"+end_label+"\n")
    

cdef whileFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    line = line + 1 # while
    line = line + 1 # (
    while_label = get_label()
    false_label = get_label()
    file1.write("label WHILE_EXP"+while_label+"\n")
    expressionfunc()
    file1.write("not\n")
    line = line + 1 # )
    line = line + 1 # {
    file1.write("if-goto WHILE_END"+false_label+"\n")
    statment()
    file1.write("goto WHILE_EXP"+while_label+"\n")
    file1.write("label WHILE_END"+false_label+"\n")
    line = line + 1 # }
    
cdef letFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    line = line + 1 # let
    varName = fileArray[line+1].split(" ")[1]
    line = line + 1
    kind1 = methodsymbolTable.kindOf(varName)
    num1 = methodsymbolTable.indexOf(varName)
    if (kind1 == None):
        kind1 = classsymbolTable.kindOf(varName)
        num1 = classsymbolTable.indexOf(varName)
    flag = (fileArray[line+1].split(" ")[1] == "[")
    if flag:
        line = line + 1 # [
        expressionfunc()
        line = line + 1 # ]
        line = line + 1 # =
        file1.write("push "+kind1+" "+num1+"\n")
        file1.write("add\n")
        expressionfunc()
        file1.write("pop temp 0\n")
        file1.write("pop pointer 1\n")
        file1.write("push temp 0\n")
        file1.write("pop that 0\n")
    else:
        line = line + 1 # =
        expressionfunc()
        file1.write("pop "+kind1+" "+num1+"\n")
    line = line + 1# ;

cdef doFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    line = line + 1 # do
    termFunc()
    file1.write("pop temp 0\n")
    line = line + 1# ;
    
cdef returnFunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    line = line + 1 # return
    if not(";" in fileArray[line+1]):
        expressionfunc()
    else:
        file1.write("push constant 0\n")
    file1.write("return\n")
    line = line + 1 # ;
        
cdef expressionfunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    termFunc()
    while (("+" in fileArray[line+1].split(" ")[1]) or ("-" in fileArray[line+1].split(" ")[1]) or ("*" in fileArray[line+1].split(" ")[1]) 
           or ("/" in fileArray[line+1].split(" ")[1]) or ("&amp" in fileArray[line+1].split(" ")[1]) or ("|" in fileArray[line+1].split(" ")[1]) 
           or ("&lt" in fileArray[line+1].split(" ")[1]) or ("&gt" in fileArray[line+1].split(" ")[1]) or ("=" in fileArray[line+1].split(" ")[1])):
        line = line + 1
        temp = fileArray[line].split(" ")[1]
        termFunc()
        if ("+" in temp):
            file1.write("add\n")
        elif ("-" in temp):
            file1.write("sub\n")
        elif ("*" in temp):
            file1.write("call Math.multiply 2\n")
        elif ("/" in temp):
            file1.write("call Math.divide 2\n")
        elif ("&amp" in temp):
            file1.write("and\n")
        elif ("|" in temp):
            file1.write("or\n")
        elif ("&gt;" in temp):
            file1.write("gt\n")
        elif ("&lt;" in temp):
            file1.write("lt\n")
        elif ("=" in temp): 
            file1.write("eq\n")
        
cdef termFunc():
    global fileArray
    global file1
    global line
    global subroutinType
    global methodsymbolTable
    global classsymbolTable
    cdef str tempTT
    line = line + 1
    if ("stringConstant" in fileArray[line]):
        file1.write("push constant ")
        ezer = fileArray[line].split(">")[1]
        ezer = ezer.split("<")
        ezer = ezer[0].lstrip()[:-1]
        lengthTemp = len(ezer)
        tempTT = str(lengthTemp)
        file1.write(tempTT)
        file1.write("\n")
        file1.write("call String.new 1\n")
        for i in range(lengthTemp):
            tempStr = ord((ezer)[i])
            file1.write("push constant ")
            tempTT = str(tempStr)
            file1.write(tempTT)
            file1.write("\n")
            file1.write("call String.appendChar 2\n") 
    elif ("-" in fileArray[line]) or ("~" in fileArray[line]):
        tempOp = line
        termFunc()
        if ("-" in fileArray[tempOp]):
            file1.write("neg\n")
        elif ("~" in fileArray[tempOp]):
            file1.write("not\n")
    elif ("(" in fileArray[line]):
        expressionfunc()
        line = line + 1
    elif("integerConstant" in fileArray[line]):
        file1.write("push constant ")
        file1.write(fileArray[line].split(" ")[1])
        file1.write("\n")
    elif ("false" in fileArray[line]) or ("null" in fileArray[line]):
        file1.write("push constant 0\n")
    elif ("true" in fileArray[line]):
        file1.write("push constant 0\n")
        file1.write("not\n")
    elif ("this" in fileArray[line]):
        file1.write("push pointer 0\n")
    elif ("identifier" in fileArray[line]):
        if ("[" in fileArray[line+1]):
            line = line + 1 # [
            current = line - 1
            expressionfunc()
            kind1 = methodsymbolTable.kindOf(fileArray[current].split(" ")[1])
            num1 = methodsymbolTable.indexOf(fileArray[current].split(" ")[1])
            if (kind1 == None):
                kind1 = classsymbolTable.kindOf(fileArray[current].split(" ")[1])
                num1 = classsymbolTable.indexOf(fileArray[current].split(" ")[1])
            file1.write("push "+kind1+" "+num1+"\n")
            file1.write("add\n")
            file1.write("pop pointer 1\n")
            file1.write("push that 0\n")
            line = line + 1 #] 
        else:
            funcName = fileArray[line].split(" ")[1]
            funcClass = fileArray[2].split(" ")[1]
            defaultCall = True
            argCount = 0
            flag2 = True
            if ("." in fileArray[line+1]):
                defaultCall = False
                line = line + 1 # .
                kind1 = methodsymbolTable.kindOf(funcName)
                num1 = methodsymbolTable.indexOf(funcName)
                if (kind1=="local"):
                    file1.write("push "+kind1+" "+num1+"\n") 
                    argCount +=1
                kind1 = classsymbolTable.kindOf(funcName)
                num1 = classsymbolTable.indexOf(funcName)
                if (kind1=="this"):
                    file1.write("push "+kind1+" "+num1+"\n") 
                    argCount +=1
                type1 = methodsymbolTable.typeOf(funcName)
                if (type1 == None):
                    type1 = classsymbolTable.typeOf(funcName)
                if not(type1 == None):
                    funcClass = type1
                    flag2 = False
                line = line + 1 # funcName
                className = fileArray[2].split(" ")[1] 
                if (funcName == className):
                    if (subroutinType == "mathod"):
                        argCount += 1
                        file1.write("push this 0\n")
                    elif (subroutinType == "function"):
                        argCount += 0
                    funcName = fileArray[line].split(" ")[1]
                else:
                    funcName = fileArray[line].split(" ")[1]
                    if (flag2):
                        funcClass = fileArray[line-2].split(" ")[1]
            if ("(" in fileArray[line+1]):
                if (defaultCall):
                    argCount = 1
                    file1.write("push pointer 0\n")
                line = line + 1 # (
                argCount += expressionListfunc()
                file1.write("call "+funcClass+"."+funcName+" "+str(argCount)+"\n")
                line = line + 1 # )
            else:
                kind1 = methodsymbolTable.kindOf(fileArray[line].split(" ")[1])
                num1 = methodsymbolTable.indexOf(fileArray[line].split(" ")[1])
                if (kind1 == None):
                    kind1 = classsymbolTable.kindOf(fileArray[line].split(" ")[1])
                    num1 = classsymbolTable.indexOf(fileArray[line].split(" ")[1])
                file1.write("push "+kind1+" "+num1+"\n")          

cdef expressionListfunc():
    global fileArray
    global file1
    global line
    global methodsymbolTable
    global classsymbolTable
    count = 0
    while not(")" in fileArray[line+1]):
        if (" , " in fileArray[line+1]):
            line = line + 1
        count = count + 1
        expressionfunc()
    return count

#MAIN ------------------------------------------------------------------- MAIN
#MAIN ------------------------------------------------------------------- MAIN            
#MAIN ------------------------------------------------------------------- MAIN
path = input()
print("\n")

global fileArray
global file1
global line
global methodsymbolTable
global classsymbolTable

for filenames in os.listdir(path):
    if("T.xml" in filenames):
        fileArray = []
        file1 = 0
        line = 2
        with open(os.path.join(path, filenames)) as file:
            strt = (file.name).split("\\")[-1]
            if("T.xml" in strt):
                line = 2
                strt1 = (filenames).split(".")
                temp = path+"\\"
                file1 = open(temp+strt1[0][:-1]+".vm", "w")
                fileArray = file.readlines();
                classFunc()
                #classsymbolTable.printMyTable()
                subroutinfunc()
                line = line + 1 # }
                file1.close()

C:\Users\איילת\Desktop\עקרונות שפות תוכנה\nand2tetris\tools\JackSnake\New folder (2)


